In [15]:
import numpy as np
from copy import deepcopy
import pickle

class ID3DTree:
    def __init__(self):
        # 构造的树
        self.tree = {}
        # 数据集
        self.dataSet = []
        # 特征集的标签名字
        self.labels = []
        
    def loadDataSet(self,path,feature_names):
        recordlist = []
        fp = open(path,'rb')
        content = fp.read().decode('utf-8')
        fp.close()
        rowlist = content.splitlines() 	# 按行转换为一维表
        recordlist = [row.split('\t') for row in rowlist if row.strip()]
        self.dataSet = recordlist
        self.feature_names = feature_names
        return self.dataSet,feature_names
        
    def train(self):
        feature_names = deepcopy(self.feature_names)
        self.tree = self.buildTree(self.dataSet,feature_names)
    
    def buildTree(self,dataSet,feature_names):
        # 抽取源数据集的决策分类标签列
        catelist = [data[-1] for data in dataSet]
        
        # 终止条件1：只有一个类别标签返回这个决策标签
        if catelist.count(catelist[0]) == len(catelist):
            return catelist[0]
        # 终止条件2：如果没有特征可以划分，返回决策标签中最多的一类
        if len(dataSet[0]) == 1:
            return self.maxCate(catelist)
        
        # 算法核心
        # 获取最好的特征所在的列
        bestFeatureIndex = self.getBestFeature(dataSet)
        bestFeatureName = feature_names[bestFeatureIndex]
        tree = {bestFeatureName:{}}
        del feature_names[bestFeatureIndex]
        
        # 抽取最优特征轴的列向量,及该最优特征有多少个属性值
        uniqueVals = set([data[bestFeatureIndex] for data in dataSet])
        # 决策树递归生长
        for value in uniqueVals:
            # 将删除后的特征类别集建立子类别集
            sub_feature_names = feature_names[:]
            # 按最优特征列和值分割数据集
            splitDataSet = self.splitDataSet(dataSet,bestFeatureIndex,value)
            # 构建子树 
            subTree = self.buildTree(splitDataSet,sub_feature_names)
            tree[bestFeatureName][value] = subTree
        return tree
     
    # 计算数量最多的类别标签
    def maxCate(self,catelist):
        items = dict([(catelist.count(i),i) for i in catelist])
        return items[max(items.keys())]
    
    # 计算最优特征所在的列
    def getBestFeature(self,dataSet):
        # 计算特征向量的维数
        numFeatures = len(dataSet[0]) - 1
        # 计算基础熵 H(D)
        BaseEntropy = self.computeEntropy(dataSet)
        # 初始化最好的信息增益
        bestInfoGain = 0.0
        # 初始化最优特征轴
        bestFeatureIndex = -1
        # 外循环：遍历数据集各列,计算最优特征轴; i为数据集列索引：取值范围 0~(numFeatures-1)
        for i in range(numFeatures):
            # 存储特征的属性值
            uniqueVals = set([data[i] for data in dataSet])
            # 初始化该列的香农熵
            newEntropy = 0.0
            # 内循环：按特征列和特征属性计算香农熵
            for value in uniqueVals:
                # 按选定列i和唯一值分隔数据集
                subDataSet = self.splitDataSet(dataSet, i, value)
                # 该特征属性在该特征中占比
                prob = len(subDataSet)/float(len(dataSet))
                # 计算在已知某特征的情况下的信息熵H（D|A）,
                newEntropy += prob * self.computeEntropy(subDataSet)
            # 计算信息增益
            InfoGain = BaseEntropy - newEntropy
            # 信息增益大于当前最好的信息增益，将其赋值给bestInfoGain,并记录最大信息增益对应的特征轴
            if InfoGain > bestInfoGain:
                bestInfoGain = InfoGain
                bestFeatureIndex = i
                
        return bestFeatureIndex
    
    # 计算信息熵
    def computeEntropy(self,dataSet):
        datalen = float(len(dataSet))
        # 抽取数据集的类别列
        catelist = [data[-1] for data in dataSet]
        # 统计每一类出现次数
        items = dict([(i,catelist.count(i)) for i in catelist])
        # 初始化信息熵
        InfoEntropy = 0.0
        # 计算信息熵
        for key in items:
            prob = items[key]/datalen
            InfoEntropy -= prob * np.log2(prob)
            
        return InfoEntropy
    
    # 划分数据集，dataSet：数据集; axis：特征轴; value：特征轴的取值
    def splitDataSet(self,DataSet,axis,value):
        subDataSet = []
        for feature_vector in DataSet:
            if feature_vector[axis] == value:
                sub_feature_vector = feature_vector[:axis]
                sub_feature_vector.extend(feature_vector[axis+1:])
                subDataSet.append(sub_feature_vector)
        return subDataSet
    
    # 数据预测
    def predict(self,InputTree,feature_names,testdata):
        # 根节点
        root = list(InputTree.keys())[0]
        # value-子树结构或分类标签
        secondDict = InputTree[root]  
        # 节点在特征中的位置
        feature_index = feature_names.index(root)
        # 节点的特征属性取值
        key = testdata[feature_index]
        # 该特征取值对应的子树结构或分类标签
        valueOfFeat = secondDict[key] 
        # 判断取值为子树结构还是标签
        if isinstance(valueOfFeat,dict):
            classLabel = self.predict(valueOfFeat,feature_names,testdata)
        else:
            classLabel = valueOfFeat
        return classLabel
    
    # 持久化
    # 存储树到文件
    def storeTree(self,inputTree,filename):
        fw = open(filename,'wb')
        pickle.dump(inputTree,fw)
        fw.close()
        
    # 从文件抓取树    
    def grabTree(self,filename):
        fr = open(filename,'rb')
        tree = pickle.load(fr)
        fr.close()
        return tree 

In [16]:
# -*- coding: utf-8 -*- 
from numpy import * 
# import treePlotter2

# 创建决策树
dtree = ID3DTree() 
# ["age","revenue","student","credit"] 对应 年龄、收入、学生、信誉 四个特征
dataSet,feature_names = dtree.loadDataSet("dataset.dat",["age","revenue","student","credit"]) 
dtree.train() 
print(dtree.tree)
print(shape(dataSet))

{'age': {'1': 'yes', '2': {'credit': {'1': 'no', '0': 'yes'}}, '0': {'student': {'1': 'yes', '0': 'no'}}}}
(1024, 5)


In [26]:
# 决策树持久化
dtree.storeTree(dtree.tree,"ID3DTree")
mytree = dtree.grabTree("ID3DTree")
print(mytree)

{'age': {'1': 'yes', '2': {'credit': {'1': 'no', '0': 'yes'}}, '0': {'student': {'1': 'yes', '0': 'no'}}}}


In [27]:
# 使用决策树预测
labels = ["age","revenue","student","credit"]
vector = ['2','0','0','1']
print("真实输出 ","no","->","决策树输出",dtree.predict(mytree,labels,vector))

真实输出  no -> 决策树输出 no
